In [7]:
# imports for conditionSetup
from __future__ import division  # so that 1/3=0.333 instead of 1/3=0
import numpy as np
from psychopy import data
from psychopy.tools import coordinatetools as coord
import os
from os.path import expanduser
import sys
import cPickle as pickle

### Assign Random Seed ###
assert len(sys.argv) == 3, "Too many inputs"
assert sys.argv[1].isdigit() and sys.argv[2].isdigit(), "Integers necessary"
subjHash = (int(sys.argv[1]), int(sys.argv[2]))
np.random.seed(seed=subjHash)

############# Object Location Rules ########################
# maxNumObjsPerQuadrant    = 2
# maxNumTargersPerQuadrant = 1
# maxNumObjsPerRadix       = 4
############################################################

####### Constants ######
nTrials         = 368
SubjDistance = 100 # distance from the screen in centimeters
MonitorWidthCM  = 27.94  # in cm

# Targets and distractors sets
TargetSets      = np.array([2,4])
DistractorSets  = np.array([0,2,4])
# Set orientations and location for each item
PossibleObjOrientations  = np.concatenate((np.arange(10,81),np.arange(100,171)))
PossibleObjRadix         = np.array([2, 4])
PossibleObjTheta         = np.vstack((np.arange(20,71),np.arange(110,161), \
                                           np.arange(200,251),np.arange(290,341)))
nConds = TargetSets.size*DistractorSets.size

# dictionary for # of trials and # of subcondtions
nTrialsPerCond       = {1:44, 2:96, 3:44, 4:50, 5:84 ,6:50}
nSubCondsPerCond     = {1:4,2:16,3:4,4:1,5:4,6:1}
assert sum(nTrialsPerCond.values()) == nTrials
# eg. conditon 1 has 4 subconditions related target/distractor positioning

# integer division for number of trials per subcondition
nTrialsPerSubCond    = {}
for cond in nTrialsPerCond.keys():
    assert nTrialsPerCond[cond] % nSubCondsPerCond[cond] == 0, "nTrialsPerCond not divisible by nSubCondsPerCond"
    nTrialsPerSubCond[cond] = int(nTrialsPerCond[cond]/nSubCondsPerCond[cond])

# make filename based on date
date = data.getDateStr()
homeDirectory = expanduser("~")
filename = homeDirectory + os.sep + 'vwmtest/data/vwm_setup_' + date
pickleFilename = homeDirectory + os.sep + "vwmtest/subj" + sys.argv[1] + "_run" + sys.argv[2] + ".p"

# An ExperimentHandler isn't essential but helps with data saving
thisExp = data.ExperimentHandler(name='setup', version='', runtimeInfo=None,
    originPath=None,
    savePickle=True, saveWideText=True,
    dataFileName=filename)

######### Assign Condition Names ###############
conds               = {}
nTotalObjsPerCond   = {}
nTargetsInCond      = {}
nDistractorsInCond  = {}
cnt = 1;
for ts in range(TargetSets.size):
    for ds in range(DistractorSets.size):
        conds[cnt]              = 'nT' + str(TargetSets[ts]) + 'nD' + str(DistractorSets[ds])
        nTargetsInCond[cnt]     = TargetSets[ts]
        nDistractorsInCond[cnt] = DistractorSets[ds]
        nTotalObjsPerCond[cnt]  = int(nTargetsInCond[cnt]+nDistractorsInCond[cnt])
        cnt += 1

###  Visual Working Memory Object Class
class VWMObj():
    """ Trial objects. """
    size = (.5, 1.5)
    def __init__(self, objType, objID , centerLoc = (0,0), orientation = 0):
    # center location tuple: (radii in deg from center of screen, theta from median)
        if objType=="target":
            self.color      = 'firebrick'
        elif objType=="distractor":
            self.color      = 'mediumblue'
        else:
            self.color      = 'black'

        self.centerLoc      = centerLoc
        self.orientation    = orientation
        self.objID          = objID
        self.objType        = objType

    def getLoc(self):
        return self.centerLoc

    def getOrientation(self):
        return self.orientation

    def getObjID(self):
        return self.objID

    def getColor(self):
        return self.color

    def objType(self):
        return self.objType

    def setOrientation(o):
        self.orientation = o

    def setLoc(p):
        self.centerLoc = p

###  Visual Working Memory Trial Class
class VWMTrial():
    """trial properties for VWM alpha tACS """
    def __init__(self, trialID, condNum, Change):
        self.trialID        = trialID
        self.condNum        = condNum
        self.nTargets       = nTargetsInCond[condNum]
        self.nDistractors   = nDistractorsInCond[condNum]
        self.nTotalItems    = self.nDistractors + self.nTargets
        self.ChangeTrial    = Change
        self.ChangeTargID   = 1
        self.ChangeTargSign = 0
        self.Objects        = []
        self.ObjTarg        = np.zeros(self.nTotalItems,np.int)
        cnt = 0
        for obj in range(self.nTargets):
            self.Objects.append(VWMObj('target',cnt))
            self.ObjTarg[cnt]=1
            cnt+=1
        for obj in range(self.nDistractors):
            self.Objects.append(VWMObj('distractor',cnt))
            self.ObjTarg[cnt]=0
            cnt+=1

# sub-condition target/distractors pairings; note that these are all indexed/key by condition
CondsTargetQuadrants = {}
CondsDistraQuadrants = {}

# Quadrant pairings and groupings for a trial
allquads        		= np.array([1,2,3,4],np.int)
quadPairings            = np.array([[1,3],[2,4],[2,3],[1,4]],np.int) # avoids (1,2), (3,4) pairings
for cond in conds:
    nn = nSubCondsPerCond[cond]
    if cond==1: # 2 targets 0 distractors (4 sub-conditions)
        CondsTargetQuadrants[cond]  = quadPairings
        CondsDistraQuadrants[cond]  = np.zeros([nn,2],np.int)
    elif cond==2: # 2 targets 2 distractors (16 sub-conditions)
        n=quadPairings.shape[0];
        CondsTargetQuadrants[cond] = np.zeros([nn,2],np.int)
        CondsDistraQuadrants[cond] = np.zeros([nn,2],np.int)
        subcond = 0
        for jj in range(n):
            for ii in range(n):
                CondsTargetQuadrants[cond][subcond] = quadPairings[jj]
                CondsDistraQuadrants[cond][subcond] = quadPairings[ii]
                subcond +=1
    elif cond==3: # 2 targets 4 distractors (4 sub-conditions)
        CondsTargetQuadrants[cond] = np.array(quadPairings)
        CondsDistraQuadrants[cond] = np.tile(allquads,(4,1))
    elif cond==4: # 4 targets 0 distractors (1 sub-conditions)
        CondsTargetQuadrants[cond] = np.array([allquads])
        CondsDistraQuadrants[cond] = np.zeros([nn,2],np.int)
    elif cond==5: # 4 targets, 2 distractors (4 sub-conditions)
        CondsTargetQuadrants[cond] = np.tile(allquads,(4,1))
        CondsDistraQuadrants[cond] = np.array(quadPairings)
    elif cond==6: # 4 targets 4 distractors (1 sub-conditions)
        CondsTargetQuadrants[cond] = np.array([allquads])
        CondsDistraQuadrants[cond] = np.array([allquads])

# Set trial conditions and counterbalance
TrialIDs        = np.arange(nTrials)  # trial IDs
AvailableTrials = np.array(TrialIDs,np.int)
TrialCondIDs    = np.zeros(nTrials,np.int)   # individual trial condition
TrialSubCondID  = np.zeros(nTrials,np.int)   #
ChangeTrialIDs  = np.zeros(nTrials,np.int)   # test array changes at test
nTrialObjs      = np.zeros(nTrials,np.int)   # total number of trials per item

# balance subconditions
for cond in conds:
    # assign condition to trials
    trials = np.random.choice(AvailableTrials,nTrialsPerCond[cond],replace=False)
    TrialCondIDs[trials] = cond
    ChangeTrialIDs[np.random.choice(trials,int(nTrialsPerCond[cond]/2),replace=False)] = 1
    nTrialObjs[trials] = nTotalObjsPerCond[cond]

    # assign subconditions to trials
    AvailableTrials2 = np.array(trials)
    for subcond in range(nSubCondsPerCond[cond]):
        trials2 = np.random.choice(AvailableTrials2,nTrialsPerSubCond[cond],replace=False)
        TrialSubCondID[trials2]= subcond+1
        AvailableTrials2 = np.setxor1d(AvailableTrials2,trials2)

    AvailableTrials = np.setxor1d(AvailableTrials,trials)

# set center positions, check for overlap
ObjectPositions = {}
TargetPos       = {}
DistractorPos   = {}
for tt in range(nTrials):
    cond        = int(TrialCondIDs[tt])
    subCond     = int(TrialSubCondID[tt])-1
    nT          = nTargetsInCond[cond]
    nD          = nDistractorsInCond[cond]

    TargsQuad   = CondsTargetQuadrants[cond][subCond]
    DistracQuad = CondsDistraQuadrants[cond][subCond]

    trialTargPos = []
    targRadPos   = np.empty(4)
    targRadPos[:] =np.nan
    for ii in range(nT):
        qq = TargsQuad[ii] # target quadrant
        theta = np.random.choice(PossibleObjTheta[qq-1], replace=False)
        radixID = int(np.random.random()>0.5)
        targRadPos[qq-1] = radixID
        radix = PossibleObjRadix[radixID]
        x,y = coord.pol2cart(theta, radix, units='deg')
        trialTargPos.append((x,y))

    trialDisPos = []
    for ii in range(nD):
        qq = DistracQuad[ii] # target quadrant
        theta = np.random.choice(PossibleObjTheta[qq-1], replace=False)
        if targRadPos[qq-1]==1:
            radix = PossibleObjRadix[0]
        elif targRadPos[qq-1]==0:
            radix = PossibleObjRadix[1]
        else:
            radix = PossibleObjRadix[int(np.random.random()>0.5)]

        x,y = coord.pol2cart(theta, radix, units='deg')
        trialDisPos.append((x,y))

    TargetPos[tt] = trialTargPos
    DistractorPos[tt] = trialDisPos

# set orietantions
# orientations sampled at random from PossibleOrientations array
ObjectOrietations = {};
for tt in range(nTrials):
    ObjectOrietations[tt] = np.random.choice(PossibleObjOrientations,nTrialObjs[tt])


saved data to u'/Users/jonaheisenberg/vwmtest/data/vwm_setup_2016_Jun_29_2356.csv'


In [8]:
ObjectOrientations[1]

NameError: name 'ObjectOrientations' is not defined

In [1]:
import numpy as np

In [6]:
range(3,28) + range(33,58) + range(63,88) + range(93,118) + range(123,148) + range(153,178)

[3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177]